In [2]:
# Install required packages
%pip install langchain
%pip install langchain_community
%pip install unstructured
%pip install langchain_openai
%pip install langchain_groq
%pip install langchain_pinecone
%pip install python-magic-bin
%pip install python-dotenv
%pip install rank_bm25

import os
import json
import tiktoken
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


c:\Users\Murshed Jamil Alif\Desktop\GenAI\ACE SYSTEM\Exp5\test6\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Murshed Jamil Alif\Desktop\GenAI\ACE SYSTEM\Exp5\test6\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pineco

In [3]:
load_dotenv()

# Initialize tokenizer to count tokens
tokenizer = tiktoken.get_encoding("cl100k_base")

def count_tokens(text):
    """Count tokens in text using tiktoken"""
    return len(tokenizer.encode(text))

In [4]:
# ==========================================
# STEP 1: Load Documents
# ==========================================

# Load all text files from directory
dir_loader = DirectoryLoader(
    "r",
    glob="**/*.txt",  
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=True
)

documents = dir_loader.load() 

print(f"Loaded {len(documents)} documents")
for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"  Source: {doc.metadata['source']}")
    print(f"  Length: {len(doc.page_content)} characters")
    print(f"  Tokens: {count_tokens(doc.page_content)}")

100%|██████████| 1/1 [00:04<00:00,  4.13s/it]

Loaded 1 documents

Document 1:
  Source: r\04. Income Tax Alternative Dispute Resolution Rules, 2024_complete_transcription.txt
  Length: 43280 characters
  Tokens: 43810


In [5]:
# ==========================================
# STEP 2: Setup OpenAI Embeddings & LLM for Chunking
# ==========================================

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    api_key=os.getenv("OPENAI_API_KEY")
)

# Initialize LLM for chunking (using OpenAI for better instruction following)
chunking_llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4.1",  # Use GPT-4 for better chunking quality
    temperature=0.5,
    max_tokens=None
)

print("OpenAI Embedding Model and Chunking LLM loaded successfully!")

# Test embeddings
query_result = embeddings.embed_query("Hello world")
print("Embedding dimension:", len(query_result))

OpenAI Embedding Model and Chunking LLM loaded successfully!
Embedding dimension: 3072


In [ ]:
# ==========================================
# STEP 3: LLM-Based Intelligent Chunking (FIXED)
# ==========================================

# Create the system prompt for LLM-based chunking (FIXED - Escaped curly braces)
chunking_system_prompt = """
You are an expert in Bangladeshi legal document structure and text processing. Your task is to intelligently chunk legal documents while preserving their hierarchical structure and semantic meaning.

# CHUNKING GUIDELINES (STRUCTURE-First)
1) Preserve Legal Structure
- Segment by legal units: “বিধি <number>” (Rule), “উপ-বিধি”, “দফা”, “উপ-দফা”, Definitions, Procedures, Penalties, Annex/Schedule, Tables (সারণী), Header, Sign-off.
- Do not break units across spans.

2) Tables (সারণী) — DO NOT SPLIT
- Keep entire table (caption, headers, body, notes) in ONE span.
- Add table-linkage metadata (e.g., linked Rule/Section).

3) Coherent Units
- One span per complete unit (e.g., Rule + its উপ-বিধি/দফা + tables/notes).
- Keep amendment directives + context in one span.
- Merge small adjacent units to avoid tiny spans.
- Avoid extreme span size imbalance.

4) Header & Sign-off
- Merge short header with following block if small.
- Separate sign-off/publishing info only if sizable.

5) Cross-References
- Capture cross-references (e.g., “ধারা ৩৪৩ / Section 343”) in metadata.
- Include span titles (e.g., Rule headings) in metadata.

# STRUCTURE CUES
- Titles: “উৎসে কর বিধিমালা, ২০২৪”, “TDS Rules, 2024”
- Units: “বিধি <number>”, “উপ-বিধি”, “দফা”, “উপ-দফা”
- Tables/Annex: “সারণী”, “পরিশিষ্ট”, “Schedule”, “Appendix”
- Cross-refs: “ধারা ৩৪৩ / Section 343”, other Acts/Rules
- Amendments: Date, SRO numbers, “সংশোধন / Amendment”

# METADATA (COMPACT, PINECONE-SAFE)
All values must be strings, numbers, booleans, or list[str]. Omit keys if unknown.
Required keys for every span:
- doc_type: "RULES"
- span_unit: ["header", "rule", "subrule-group", "table", "annex", "definitions", "amendment", "procedures", "penalties", "signoff", "mixed"]
- span_title: Short heading/label (e.g., “বিধি ৫”)
- rules_name: e.g., “TDS Rules, 2024”
- rules_year: e.g., “2024”
- language: “bn” or “bn+en”
- is_amendment: true/false

Optional keys (include where applicable):
- rule_numbers: list[str], e.g., [“বিধি ৫”, “Rule 5”]
- section_refs: list[str], e.g., [“ধারা ৩৪৩”, “Section 343”]
- span_has_table: true/false
- table_ids: list[str], e.g., [“সারণী-১”, “Table-1”]
- table_linked_rule_numbers: list[str], e.g., [“বিধি ৫”]
- keywords: list[str]

# OUTPUT FORMAT (JSON ONLY)
Return strictly valid JSON, no markdown fences, no commentary. Each chunk MUST include a `content` string that contains the exact text of the span. If you compute indices, ensure `content = document_text[start:end]`.

{{
  "chunks": [
    {{
      "start": 0,
      "end": 1234,
      "metadata": {{
        "doc_type": "RULES",
        "span_unit": "rule",
        "span_title": "বিধি ৫",
        "rules_name": "TDS Rules, 2024",
        "rules_year": "2024",
        "language": "bn+en",
        "is_amendment": false,
        "rule_numbers": ["বিধি ৫", "Rule 5"],
        "section_refs": ["ধারা ৩৪৩"],
        "span_has_table": false,
        "table_ids": [],
        "table_linked_rule_numbers": [],
        "keywords": ["বিধি ৫", "Rule 5", "ধারা ৩৪৩", "TDS", "উৎসে কর", ...]
      }}
    }}
  ]
}}

**Quality Checks:**
- Ensure no chunk exceeds 4000 tokens (STRICT).
- Ensure no critical legal provision is split.
- Maintain readability and coherence.
- Include sufficient context for understanding.

Now process the following legal document:
"""


def llm_chunk_document(document: Document, max_retries: int = 2) -> List[Document]:
    """
    Use LLM to intelligently chunk a legal document
    """
    print(f"\nProcessing document: {document.metadata.get('source', 'Unknown')}")
    
    # Create the prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", chunking_system_prompt),
        ("human", "Document to chunk:\n\n{document_text}")
    ])
    
    # Chain LLM with prompt
    chunking_chain = prompt | chunking_llm
    
    for attempt in range(max_retries + 1):
        try:
            print(f"  Attempt {attempt + 1} - Sending to LLM for chunking...")
            
            # Get LLM response
            response = chunking_chain.invoke({
                "document_text": document.page_content
            })
            
            # Parse JSON response
            response_text = response.content.strip()
            
            # Clean up the response (remove markdown formatting if present)
            if response_text.startswith("```json"):
                response_text = response_text[7:]
            if response_text.endswith("```"):
                response_text = response_text[:-3]
            
            # Parse JSON
            chunks_data = json.loads(response_text)
            
            # Create Document objects
            chunk_documents = []
            for i, chunk_info in enumerate(chunks_data.get("chunks", [])):
                # Validate chunk size
                chunk_content = chunk_info.get("content", "")
                chunk_tokens = count_tokens(chunk_content)
                
                if chunk_tokens > 3000:
                    print(f"    Warning: Chunk {i+1} is {chunk_tokens} tokens (>3000)")
                
                # Create metadata
                chunk_metadata = document.metadata.copy()
                chunk_metadata.update(chunk_info.get("metadata", {}))
                chunk_metadata["chunk_index"] = i
                chunk_metadata["total_chunks"] = len(chunks_data.get("chunks", []))
                chunk_metadata["chunk_tokens"] = chunk_tokens
                
                # Create Document
                chunk_doc = Document(
                    page_content=chunk_content,
                    metadata=chunk_metadata
                )
                chunk_documents.append(chunk_doc)
            
            print(f"  ✅ Successfully created {len(chunk_documents)} chunks")
            
            # Print chunk statistics
            for i, chunk in enumerate(chunk_documents):
                tokens = chunk.metadata.get("chunk_tokens", 0)
                chunk_type = chunk.metadata.get("chunk_type", "unknown")
                print(f"    Chunk {i+1}: {tokens} tokens, type: {chunk_type}")
            
            return chunk_documents
            
        except json.JSONDecodeError as e:
            print(f"    ❌ JSON parsing error on attempt {attempt + 1}: {e}")
            if attempt == max_retries:
                print(f"    ❌ All attempts failed - skipping document")
                return []
            
        except Exception as e:
            print(f"    ❌ Error on attempt {attempt + 1}: {e}")
            if attempt == max_retries:
                print(f"    ❌ All attempts failed - skipping document")
                return []
    
    return []

def process_all_documents_with_llm(documents: List[Document]) -> List[Document]:
    """
    Process all documents using LLM-based chunking
    """
    all_chunks = []
    
    print(f"\n🚀 Starting LLM-based chunking for {len(documents)} documents...")
    
    for i, doc in enumerate(documents):
        print(f"\n--- Processing Document {i+1}/{len(documents)} ---")
        
        # Check document size
        doc_tokens = count_tokens(doc.page_content)
        print(f"Document tokens: {doc_tokens}")
        
        if doc_tokens < 100:
            print("  ⚠️  Document too small, skipping...")
            continue
            
        # Process with LLM
        doc_chunks = llm_chunk_document(doc)
        all_chunks.extend(doc_chunks)
        
        print(f"  📊 Total chunks so far: {len(all_chunks)}")
    
    return all_chunks

# Process documents with LLM-based chunking
print("\n🤖 Starting LLM-based intelligent chunking...")
chunks = process_all_documents_with_llm(documents)

print(f"\n✅ LLM Chunking Complete!")
print(f"📊 Total chunks created: {len(chunks)}")
print(f"📝 Sample chunk metadata: {chunks[0].metadata if chunks else 'No chunks'}")


🤖 Starting LLM-based intelligent chunking...

🚀 Starting LLM-based chunking for 1 documents...

--- Processing Document 1/1 ---
Document tokens: 43810

Processing document: r\04. Income Tax Alternative Dispute Resolution Rules, 2024_complete_transcription.txt
  Attempt 1 - Sending to LLM for chunking...
  ✅ Successfully created 27 chunks
    Chunk 1: 674 tokens, type: unknown
    Chunk 2: 1419 tokens, type: unknown
    Chunk 3: 1390 tokens, type: unknown
    Chunk 4: 2559 tokens, type: unknown
    Chunk 5: 2680 tokens, type: unknown
    Chunk 6: 6896 tokens, type: unknown
    Chunk 7: 805 tokens, type: unknown
    Chunk 8: 973 tokens, type: unknown
    Chunk 9: 396 tokens, type: unknown
    Chunk 10: 1084 tokens, type: unknown
    Chunk 11: 673 tokens, type: unknown
    Chunk 12: 524 tokens, type: unknown
    Chunk 13: 885 tokens, type: unknown
    Chunk 14: 595 tokens, type: unknown
    Chunk 15: 827 tokens, type: unknown
    Chunk 16: 874 tokens, type: unknown
    Chunk 17: 562 toke

In [7]:
print(len(chunks))

27


In [8]:
for i in range(len(chunks)):
    print("===============CHUNK===============",i)
    print(chunks[i])

===============CHUNK=============== 0
page_content='বাংলাদেশ

অতিরিক্ত সংখ্যা

কর্তৃপক্ষ কর্তৃক প্রকাশিত

বৃহস্পতিবার, জুন ২৭, ২০২৪

গণপ্রজাতন্ত্রী বাংলাদেশ সরকার

জাতীয় রাজস্ব বোর্ড

(আয়কর)

প্রজ্ঞাপন

তারখি: ১৩ আষঢ়, ১৪৩১ বঙ্গাব্দ/২৭ জুন, ২০২৪ খ্রিষ্টাব্দ

এস.আর.ও. নং ২৪৩-আইন/আয়কর-৩৭/২০২৪।—জাতীয় রাজস্ব বোর্ড, আয়কর আইন, ২০২৩ (২০২৩ সনের ১২ নং আইন) এর ধারা ৩৪৩ এ প্রদত্ত ক্ষমতাবলে, নিম্নরূপ বিধিমালা প্রণয়ন করিল, যথা:—

১। শিরোনাম ও প্রর্বতন।—(১) এই বিধিমালা আয়কর বিকল্প বিরোধ নিষ্পত্তি বিধিমালা, ২০২৪ নামে অভিহিত হইবে।

(২) ইহা ১ জুলাই, ২০২৪ খ্রিষ্টাব্দ হইতে কার্যকর হইবে।' metadata={'source': 'r\\04. Income Tax Alternative Dispute Resolution Rules, 2024_complete_transcription.txt', 'doc_type': 'RULES', 'span_unit': ['header'], 'span_title': 'প্রারম্ভিক তথ্য ও শিরোনাম', 'rules_name': 'Income Tax Alternative Dispute Resolution Rules, 2024', 'rules_year': '2024', 'language': 'bn', 'is_amendment': False, 'keywords': ['প্রজ্ঞাপন', 'জাতীয় রাজস্ব বোর্ড', 'শিরোনাম', 'কার্যকর', 'ধারা ৩৪৩', 

In [9]:
# ==========================================
# STEP 4: Setup Pinecone
# ==========================================

# Set Pinecone API key
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

# Initialize Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# Check embedding dimension
test_embedding = embeddings.embed_query("test")
actual_dimension = len(test_embedding)
print(f"Actual embedding dimension: {actual_dimension}")

# Index settings
index_name = "rules-agentic-chunking"
embedding_dimension = 3072  # text-embedding-3-large dimension

# Create index if it doesn't exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Created new index: {index_name}")
else:
    print(f"Using existing index: {index_name}")

# Create vectorstore
vectorstore = PineconeVectorStore(
    index=pc.Index(index_name),
    embedding=embeddings
)

Actual embedding dimension: 3072
Using existing index: rules-agentic-chunking


In [11]:
# ==========================================
# STEP 5: Add Chunks to Vectorstore (FIXED)
# ==========================================

def sanitize_metadata_for_pinecone(metadata: dict) -> dict:
    """
    Sanitize metadata to comply with Pinecone requirements:
    - No null/None values
    - Only strings, numbers, booleans, or lists of strings
    """
    sanitized = {}
    
    for key, value in metadata.items():
        if value is None:
            # Skip null/None values entirely
            continue
        elif isinstance(value, str):
            # Keep non-empty strings
            if value.strip():
                sanitized[key] = value.strip()
        elif isinstance(value, (int, float, bool)):
            # Keep numbers and booleans
            sanitized[key] = value
        elif isinstance(value, list):
            # Clean lists - only keep non-empty strings
            clean_list = [str(item).strip() for item in value if item is not None and str(item).strip()]
            if clean_list:
                sanitized[key] = clean_list
        elif isinstance(value, dict):
            # Skip complex nested objects
            continue
        else:
            # Convert other types to strings
            str_value = str(value).strip()
            if str_value and str_value.lower() not in ['none', 'null', '']:
                sanitized[key] = str_value
    
    # Ensure we have at least basic metadata
    if 'source' not in sanitized:
        sanitized['source'] = 'unknown'
    if 'chunk_type' not in sanitized:
        sanitized['chunk_type'] = 'general'
    
    return sanitized

def add_chunks_to_vectorstore_fixed(vectorstore, chunks, max_tokens_per_batch=200000):
    """Add LLM-chunked documents to vectorstore - NEVER SKIP ANY CHUNKS"""
    
    if not chunks:
        print("No chunks to add!")
        return
    
    print(f"📤 Adding {len(chunks)} LLM-generated chunks to vectorstore...")
    print("🧹 Sanitizing metadata for Pinecone compatibility...")
    print("🔒 ZERO LOSS POLICY: Every chunk will be uploaded with fixed metadata")
    
    # Pre-process all chunks to sanitize metadata - NEVER SKIP
    sanitized_chunks = []
    
    for i, chunk in enumerate(chunks):
        
            # Sanitize metadata - replace nulls with defaults
            clean_metadata = sanitize_metadata_for_pinecone(chunk.metadata)
            
            # Ensure content exists
            content = chunk.page_content if chunk.page_content else "Content not available"
            
            # Create new Document with clean metadata
            clean_chunk = Document(
                page_content=content,
                metadata=clean_metadata
            )
            sanitized_chunks.append(clean_chunk)
            
    print(f"  ✅ Prepared {len(sanitized_chunks)} chunks for upload (same as input: {len(chunks)})")
    
    # Verify we haven't lost any chunks
    if len(sanitized_chunks) != len(chunks):
        raise Exception(f"CRITICAL ERROR: Chunk count mismatch! Input: {len(chunks)}, Output: {len(sanitized_chunks)}")
    
    # Now proceed with batch upload - with aggressive retry logic
    current_batch = []
    current_tokens = 0
    batch_num = 1
    successful_uploads = 0
    
    for i, chunk in enumerate(sanitized_chunks):
        chunk_tokens = chunk.metadata.get("chunk_tokens", count_tokens(chunk.page_content))
        
        # Check if adding this chunk would exceed the limit
        if current_tokens + chunk_tokens > max_tokens_per_batch and current_batch:
            # Process current batch
            print(f"Processing batch {batch_num}: {len(current_batch)} chunks, {current_tokens} tokens")
            
            success = upload_batch_with_retry(vectorstore, current_batch, batch_num)
            successful_uploads += success
            
            # Reset for next batch
            current_batch = []
            current_tokens = 0
            batch_num += 1
        
        # Add chunk to current batch
        current_batch.append(chunk)
        current_tokens += chunk_tokens
        
        if (i + 1) % 20 == 0:
            print(f"  📊 Processed {i + 1}/{len(sanitized_chunks)} chunks...")
    
    # Process final batch
    if current_batch:
        print(f"Processing final batch {batch_num}: {len(current_batch)} chunks, {current_tokens} tokens")
        success = upload_batch_with_retry(vectorstore, current_batch, batch_num)
        successful_uploads += success
    
    print(f"🎉 Upload complete! Successfully added {successful_uploads}/{len(chunks)} chunks to vectorstore!")
    
    if successful_uploads != len(chunks):
        raise Exception(f"CRITICAL ERROR: Not all chunks uploaded! Expected: {len(chunks)}, Uploaded: {successful_uploads}")

def upload_batch_with_retry(vectorstore, batch, batch_num):
    """Upload batch with aggressive retry - ensure every chunk gets uploaded"""
    
    try:
        vectorstore.add_documents(batch)
        print(f"  ✅ Batch {batch_num} successful ({len(batch)} chunks)")
        return len(batch)
        
    except Exception as e:
        print(f"  ❌ Batch {batch_num} failed: {e}")
        print(f"  🔄 Switching to individual upload mode for {len(batch)} chunks...")
        
        successful_individual = 0
        
        for j, single_chunk in enumerate(batch):
            try:
                vectorstore.add_documents([single_chunk])
                successful_individual += 1
                
            except Exception as single_error:
                print(f"    ❌ Individual chunk {j+1} failed: {single_error}")
                
                # Last resort - strip metadata to absolute minimum
                try:
                    minimal_chunk = Document(
                        page_content=single_chunk.page_content,
                        metadata={
                            'source': f'emergency_chunk_{batch_num}_{j}',
                            'chunk_type': 'general'
                        }
                    )
                    vectorstore.add_documents([minimal_chunk])
                    successful_individual += 1
                    print(f"    🆘 Emergency upload successful for chunk {j+1}")
                    
                except Exception as emergency_error:
                    print(f"    💥 CRITICAL: Cannot upload chunk {j+1} even with minimal metadata: {emergency_error}")
                    print(f"    📝 Content preview: {single_chunk.page_content[:100]}...")
                    # This should never happen, but we log it for investigation
        
        print(f"  📊 Individual upload result: {successful_individual}/{len(batch)} chunks")
        return successful_individual

# Debug function to check your current chunks
def debug_chunk_metadata(chunks, num_samples=5):
    """Debug function to inspect chunk metadata"""
    print(f"🔍 Debugging metadata for {min(num_samples, len(chunks))} sample chunks:")
    
    for i, chunk in enumerate(chunks[:num_samples]):
        print(f"\nChunk {i+1} metadata:")
        for key, value in chunk.metadata.items():
            value_type = type(value).__name__
            print(f"  {key}: {value} (type: {value_type})")
            
            if value is None:
                print(f"    ❌ NULL VALUE DETECTED in '{key}' - this will cause Pinecone error!")



In [12]:
# Run this first to see what's wrong
print("🔍 Checking your chunks for metadata issues...")
debug_chunk_metadata(chunks)



🔍 Checking your chunks for metadata issues...
🔍 Debugging metadata for 5 sample chunks:

Chunk 1 metadata:
  source: r\04. Income Tax Alternative Dispute Resolution Rules, 2024_complete_transcription.txt (type: str)
  doc_type: RULES (type: str)
  span_unit: ['header'] (type: list)
  span_title: প্রারম্ভিক তথ্য ও শিরোনাম (type: str)
  rules_name: Income Tax Alternative Dispute Resolution Rules, 2024 (type: str)
  rules_year: 2024 (type: str)
  language: bn (type: str)
  is_amendment: False (type: bool)
  keywords: ['প্রজ্ঞাপন', 'জাতীয় রাজস্ব বোর্ড', 'শিরোনাম', 'কার্যকর', 'ধারা ৩৪৩', 'header', 'SRO 243', 'alternative dispute resolution'] (type: list)
  chunk_index: 0 (type: int)
  total_chunks: 27 (type: int)
  chunk_tokens: 674 (type: int)

Chunk 2 metadata:
  source: r\04. Income Tax Alternative Dispute Resolution Rules, 2024_complete_transcription.txt (type: str)
  doc_type: RULES (type: str)
  span_unit: ['definitions'] (type: list)
  span_title: সংজ্ঞা (type: str)
  rules_name: In

In [13]:
# Then use the fixed function
add_chunks_to_vectorstore_fixed(vectorstore, chunks)

📤 Adding 27 LLM-generated chunks to vectorstore...
🧹 Sanitizing metadata for Pinecone compatibility...
🔒 ZERO LOSS POLICY: Every chunk will be uploaded with fixed metadata
  ✅ Prepared 27 chunks for upload (same as input: 27)
  📊 Processed 20/27 chunks...
Processing final batch 1: 27 chunks, 43095 tokens
  ✅ Batch 1 successful (27 chunks)
🎉 Upload complete! Successfully added 27/27 chunks to vectorstore!


In [ ]:
# # ==========================================
# # STEP 5: Add Chunks to Vectorstore
# # ==========================================

# def add_chunks_to_vectorstore(vectorstore, chunks, max_tokens_per_batch=200000):
#     """Add LLM-chunked documents to vectorstore with enhanced metadata"""
    
#     if not chunks:
#         print("No chunks to add!")
#         return
    
#     current_batch = []
#     current_tokens = 0
#     batch_num = 1
    
#     print(f"📤 Adding {len(chunks)} LLM-generated chunks to vectorstore...")
    
#     for i, chunk in enumerate(chunks):
#         chunk_tokens = chunk.metadata.get("chunk_tokens", count_tokens(chunk.page_content))
        
#         # Check if adding this chunk would exceed the limit
#         if current_tokens + chunk_tokens > max_tokens_per_batch and current_batch:
#             # Process current batch
#             print(f"Processing batch {batch_num}: {len(current_batch)} chunks, {current_tokens} tokens")
            
#             try:
#                 vectorstore.add_documents(current_batch)
#                 print(f"  ✅ Batch {batch_num} successful")
#             except Exception as e:
#                 print(f"  ❌ Batch {batch_num} failed: {e}")
#                 # Try individual chunks
#                 for single_chunk in current_batch:
#                     try:
#                         vectorstore.add_documents([single_chunk])
#                     except Exception as single_error:
#                         print(f"    ❌ Single chunk failed: {single_error}")
            
#             # Reset for next batch
#             current_batch = []
#             current_tokens = 0
#             batch_num += 1
        
#         # Add chunk to current batch
#         current_batch.append(chunk)
#         current_tokens += chunk_tokens
        
#         if (i + 1) % 20 == 0:
#             print(f"  📊 Processed {i + 1}/{len(chunks)} chunks...")
    
#     # Process final batch
#     if current_batch:
#         print(f"Processing final batch {batch_num}: {len(current_batch)} chunks, {current_tokens} tokens")
#         try:
#             vectorstore.add_documents(current_batch)
#             print(f"  ✅ Final batch successful")
#         except Exception as e:
#             print(f"  ❌ Final batch failed: {e}")
#             # Try individual chunks
#             for single_chunk in current_batch:
#                 try:
#                     vectorstore.add_documents([single_chunk])
#                 except Exception as single_error:
#                     print(f"    ❌ Single chunk failed: {single_error}")
    
#     print("🎉 All LLM chunks processed and added to vectorstore!")

# # Add chunks to vectorstore
# add_chunks_to_vectorstore(vectorstore, chunks)

In [17]:
# ==========================================
# STEP 6: Setup Retrieval Chain
# ==========================================

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={'k': 15}
)

# Initialize final LLM for answering questions
# answering_llm = ChatGroq(
#     groq_api_key=os.getenv("GROQ_API_KEY"),
#     model_name="meta-llama/llama-4-scout-17b-16e-instruct",
#     temperature=0.1,
#     max_tokens=None
# )

# Initialize Groq LLM (you can also use OpenAI)
import os
from langchain_openai import ChatOpenAI
answering_llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4.1-mini",
    temperature=0.7,
    max_tokens=None
)

In [18]:


# Enhanced system prompt for the final RAG chain
enhanced_system_prompt = (
    "আপনি বাংলাদেশের আইনভিত্তিক একটি উন্নত লিগ্যাল চ্যাটবট। আপনার জ্ঞানভান্ডার LLM-ভিত্তিক স্মার্ট চাঙ্কিং দিয়ে প্রস্তুত, "
    "যা আইনি কাঠামো এবং হায়ারার্কি বজায় রেখে সংগঠিত। আপনি পাবেন:\n\n"
    "**নথি প্রকার**: আইন/অ্যাক্ট, বিধিমালা, অধ্যাদেশ, সংশোধনী, প্রজ্ঞাপন, সার্কুলার, SRO/GO/RO\n"
    "**উন্নত মেটাডেটা**: প্রতিটি চাঙ্কে আইনের নাম, ধারা পরিসীমা, অধ্যায়, মূল শব্দ, তারিখ থাকতে পারে\n\n"
    "**নির্দেশনা**:\n"
    "1) **সূত্র নির্দেশনা**: মেটাডেটা থেকে প্রাপ্ত তথ্য ব্যবহার করে সুনির্দিষ্ট রেফারেন্স দিন\n"
    "2) **ক্রস-রেফারেন্স**: সম্পর্কিত ধারা/বিধান উল্লেখ করুন যদি প্রাসঙ্গিক হয়\n"
    "3) **কাঠামোগত উত্তর**: (ক) সংক্ষিপ্ত উত্তর (খ) আইনি ভিত্তি (গ) বিস্তারিত ব্যাখ্যা (ঘ) প্রয়োগ/সতর্কতা\n"
    "4) **স্মার্ট অনুসন্ধান**: চাঙ্ক মেটাডেটা ব্যবহার করে প্রাসঙ্গিক তথ্য খুঁজুন\n"
    "5) **ভাষা**: প্রাথমিকভাবে বাংলায়, শেষে ইংরেজি সারসংক্ষেপ\n\n"
    "প্রাপ্ত স্মার্ট চাঙ্ক কনটেক্সট:\n{context}\n\n"
    "---\n"
    "You are an advanced Bangladesh Legal Assistant with LLM-enhanced chunking. Each context chunk contains "
    "intelligent metadata including act names, section ranges, keywords, and legal hierarchy. Use this enhanced "
    "Be specific according to the law.Don't give any information out of the context.If the related answer is directly present directly mention that with lease amount of modification"
)

# Create enhanced prompt
enhanced_prompt = ChatPromptTemplate.from_messages([
    ("system", enhanced_system_prompt),
    ("human", "{input}"),
])

# Create chains
question_answer_chain = create_stuff_documents_chain(answering_llm, enhanced_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("🔗 Enhanced RAG chains ready with LLM-chunked legal documents!")

🔗 Enhanced RAG chains ready with LLM-chunked legal documents!


In [19]:
response = rag_chain.invoke({"input":"উৎসে কর বিধিমালা, ২০২৪ নিয়ে জানতে চাই?" })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তর:  
উৎসে কর বিধিমালা, ২০২৪ হলো জাতীয় রাজস্ব বোর্ড কর্তৃক আয়কর আইন, ২০২৩ এর ধারা ৩৪৩ এর অধীনে প্রণীত একটি বিধিমালা যা ১ জুলাই ২০২৪ থেকে কার্যকর হবে। এটি উৎসে কর কর্তনের নিয়মাবলী নির্ধারণ করে এবং পূর্ববর্তী "উৎসে কর বিধিমালা, ২০২৩" কে রহিত করেছে।

(খ) আইনি ভিত্তি:  
- আয়কর আইন, ২০২৩ (২০২৩ সনের ১২ নং আইন) এর ধারা ৩৪৩।  
- এস.আর.ও. নং ১৬১-আইন/আয়কর-৩৬/২০২৪, তারিখ: ১৫ জ্যৈষ্ঠ, ১৪৩১ বঙ্গাব্দ/ ২৯ মে, ২০২৪।

(গ) বিস্তারিত ব্যাখ্যা:  
- উৎসে কর বিধিমালা, ২০২৪ জাতীয় রাজস্ব বোর্ড কর্তৃক ১ জুলাই ২০২৪ থেকে কার্যকর হয়েছে।  
- এই বিধিমালা আয়কর আইন, ২০২৩ এর ধারাবলী অনুসারে উৎসে কর কর্তনের সুনির্দিষ্ট নিয়ম, হার এবং পদ্ধতি নির্ধারণ করে।  
- এটি বিভিন্ন প্রকার আয় ও পণ্যের জন্য উৎসে কর কর্তনের হার নির্ধারণ করে যেমন এমএস বিলেট উৎপাদন, তেল সরবরাহ, ধান, চাল, ফল ইত্যাদি।  
- বিধিমালায় বিভিন্ন সংশোধনী ও অতিরিক্ত ধারা সন্নিবেশিত হয়েছে, যা পরবর্তীতে ২৬ মে ২০২৫ তারিখে আরও সংশোধিত হয়েছে (এস.আর.ও. নং-১৫৭-আইন/আয়কর-১২/২০২৫)।  
- বিধি ৩ এ উল্লিখিত সারণীতে উৎসে করের হার বিস্তারিতভাবে উল্

In [9]:
response = rag_chain.invoke({"input":"ওজন মাপার যন্ত্রের উপর অবচয়ের হার কত?" })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তর:  
ওজন মাপার যন্ত্রের উপর অবচয়ের হার ১০%।

(খ) আইনি ভিত্তি:  
এটি "তৃতীয় তফসিল" এর "অবচয় ভাতা, নিঃশেষ ভাতা ও অ্যামর্টাইজেসন" অংশ ১, "অবচয় ভাতা পরিগণনা" ধারা ১(১) এর সারণীতে স্পষ্টভাবে উল্লেখ আছে, যেখানে কৃষিতে ব্যবহৃত মূলধনি পরিসম্পদের অবচয়ের হার বর্ণিত হয়েছে।

(গ) বিস্তারিত ব্যাখ্যা:  
তফসিলের অংশ ১ এর অনুচ্ছেদ ১(১) এর সারণীতে ওজন মাপার যন্ত্রের অবচয়ের হার ১০% নির্ধারিত হয়েছে। অর্থাৎ, করদাতা কৃষির উদ্দেশ্যে ব্যবহৃত ওজন মাপার যন্ত্রের অবলোপিত মূল্যের ১০% হারে বার্ষিক অবচয় ভাতা গ্রহণ করতে পারবেন।

(ঘ) প্রয়োগ/সতর্কতা:  
- অবচয় ভাতা শুধুমাত্র কৃষির উদ্দেশ্যে ব্যবহৃত ওজন মাপার যন্ত্রের ক্ষেত্রে প্রযোজ্য।  
- যদি ওই যন্ত্র আয়বর্ষে সম্পূর্ণ কৃষির উদ্দেশ্যে ব্যবহার না হয়, তবে অবচয় ভাতা আনুপাতিক হারে প্রদান করা হবে (তফসিল, ধারা ১(২))।  

সূত্র:  
তৃতীয় তফসিল, অবচয় ভাতা, নিঃশেষ ভাতা ও অ্যামর্টাইজেসন, অংশ ১, অনুচ্ছেদ ১(১), সারণী — ওজন মাপার যন্ত্রের অবচয় হার ১০%।


In [10]:
response = rag_chain.invoke({"input":"কর অবকাশ প্রাপ্তির যোগ্য ভৌতকাঠামো কোনগুলো?" })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তরঃ  
কর অবকাশ প্রাপ্তির যোগ্য ভৌত অবকাঠামো হিসেবে নিম্নলিখিত সুবিধাগুলো বিবেচিত হবে: গভীর সমুদ্র বন্দর, সমুদ্র বন্দর বা নদী বন্দর; এলিভেটেড এক্সপ্রেসওয়ে; রপ্তানি প্রক্রিয়াকরণ অঞ্চল; ফ্লাইওভার; টোলরোড ও ব্রিজ; গ্যাস পাইপ লাইন; আইসিটি পার্ক, জোন বা ভিলেজ; হাইটেক পার্ক; অনুমোদিত পানি শোধনাগার; পানি সরবরাহ বা পানি নিষ্কাশন ব্যবস্থা; তরলায়িত প্রাকৃতিক গ্যাস (এলএনজি) টার্মিনাল এবং সঞ্চালন লাইন; মনোরেল ও সাবওয়ে রেলওয়ে; নবায়নযোগ্য জ্বালানি; এবং বোর্ড কর্তৃক সরকারি গেজেটে প্রজ্ঞাপন দ্বারা নির্ধারিত অন্য কোনো ভৌত-অবকাঠামো সুবিধাদি। এছাড়া, এসব সুবিধাগুলো বাংলাদেশে অবস্থিত হতে হবে এবং জুন, ২০২৪ এর মধ্যে বাণিজ্যিক উৎপাদন শুরু করতে হবে।  

(খ) আইনি ভিত্তি:  
উপরোক্ত তথ্য কর আইনের ৩ নম্বর অনুচ্ছেদ, বিশেষত কর অবকাশ প্রাপ্তির যোগ্য ভৌত অবকাঠামোসমূহ সংক্রান্ত বিধানে পাওয়া যায়।  

(গ) বিস্তারিত ব্যাখ্যা:  
আইনের ৩(১) ও (২) ধারা অনুযায়ী কর অবকাশের জন্য যোগ্য ভৌত অবকাঠামো উন্নয়নের ক্ষেত্রে বিশেষ কিছু প্রকল্প তালিকাভুক্ত করা হয়েছে। এই প্রকল্পগুলোর মধ্যে গভীর সমুদ্র বন্দর থেকে শ

In [37]:
response = rag_chain.invoke({"input":"তহবিল হইতে আয়ের ক্ষেত্রে কি পরিমান অর্থ কর থেকে অব্যাহতি পাবে?" })
answer = response.get("answer", "No answer found")
print(answer)

সংক্ষিপ্ত উত্তর:  
তহবিল হইতে আয়ের ক্ষেত্রে নির্দিষ্ট শর্তে কর থেকে অব্যাহতি পাওয়া যায়। যদি তহবিলের আয় সুদ বা অন্য নামে হয় এবং চাঁদার অংশ ব্যতীত হয়, তবে নির্দিষ্ট সীমার মধ্যে অর্থ কর থেকে অব্যাহতি পেতে পারে।  

আইনি ভিত্তি:  
ধারা ১৫৪(১) অনুযায়ী, স্বীকৃত ভবিষ্য তহবিল বা অনুমোদিত তহবিলের আয় কর থেকে অব্যাহতি পেতে পারে। বিশেষ করে, যদি তহবিলের আয় সুদ বা অন্য নামে হয় এবং চাঁদার অংশ ব্যতীত হয়, তবে নির্দিষ্ট শর্তে কর থেকে অব্যাহতি দেওয়া হয়।  

বিস্তারিত ব্যাখ্যা:  
ধারা ১৫৪(১) অনুযায়ী, স্বীকৃত বা অনুমোদিত ভবিষ্য তহবিলের আয় কর থেকে অব্যাহতি পেতে পারে। তবে, যদি সুদ বা অন্য নামে তহবিল থেকে আয় হয় এবং চাঁদার অংশ ব্যতীত হয়, তবে নির্দিষ্ট শর্তে এই আয় কর থেকে অব্যাহতি পেতে পারে।  
অর্থের পরিমাণ নির্ধারণের জন্য, যদি ক < (খ × ৩৩%) হয়, তবে ক পরিমাণ অর্থ কর থেকে অব্যাহতি পাবে। অন্যথায়, ক - (খ × ৩৩%) এর সমপরিমাণ অর্থ আয়ের সঙ্গে যুক্ত হবে।  

সতর্কতা:  
এই সুবিধা শুধুমাত্র স্বীকৃত বা অনুমোদিত তহবিলের ক্ষেত্রে প্রযোজ্য। তহবিলের ধরণ ও আয়ের প্রকৃতি অনুযায়ী নির্দিষ্ট শর্ত পূরণ করতে হবে। 

In [15]:
response = rag_chain.invoke({"input":"বনানী এলাকায় বিল্ডিং বা অ্যাপার্ট্মেন্টে বিনিয়োগের ক্ষেত্রে কর কত?" })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তর:
বনানী এলাকায় অনধিক ২০০ বর্গমিটার প্লিন্থ আয়তন বিশিষ্ট বিল্ডিং বা অ্যাপার্টমেন্টে বিনিয়োগের ক্ষেত্রে করহার প্রতি বর্গ মিটারে ৪,০০০ (চার হাজার) টাকা এবং ২০০ বর্গমিটারের অধিক হলে প্রতি বর্গ মিটারে ৬,০০০ (ছয় হাজার) টাকা।

(খ) আইনি ভিত্তি:
প্রথম তফসিল, ধারা ২৪ দ্রষ্টব্য, অংশ ১, সারণী অনুযায়ী বনানী এলাকায় বিনিয়োগকৃত বিল্ডিং বা অ্যাপার্টমেন্টের করহার নির্ধারিত হয়েছে।

(গ) বিস্তারিত ব্যাখ্যা:
১। বনানী এলাকায় অবস্থিত বিল্ডিং বা অ্যাপার্টমেন্টের জন্য নিচের করহার প্রযোজ্য হবে:

- অনধিক ২০০ বর্গমিটার প্লিন্থ আয়তন বিশিষ্ট সম্পত্তিতে প্রতি বর্গ মিটার কর ৪,০০০ টাকা (সারণী, ক্রমিক নং ১)।

- ২০০ বর্গমিটারের অধিক প্লিন্থ আয়তন বিশিষ্ট সম্পত্তিতে প্রতি বর্গ মিটার কর ৬,০০০ টাকা (সারণী, ক্রমিক নং ২)।

২। করদাতা যদি একই সিটি কর্পোরেশন এলাকায় আগেই কোনো বিল্ডিং বা অ্যাপার্টমেন্টের মালিক হন অথবা দুই বা ততোধিক সম্পত্তিতে বিনিয়োগ করেন, তবে করহার ২০% অতিরিক্ত হবে।

৩। যদি বিনিয়োগের পূর্বে কর ফাঁকি বা গোপনের নোটিশ জারি হয়, তাহলে করহার ১০০% অতিরিক্ত দিতে হবে।

৪। বিনিয়োগকৃত অর্থ যদি অপরাধমূ

In [47]:
response = rag_chain.invoke({"input":"আপিল  ট্রাইবুন্যালে আপিল করার নিয়ম কি?" })
answer = response.get("answer", "No answer found")
print(answer)

**সংক্ষিপ্ত উত্তর:**
আপিল ট্রাইব্যুনালে আপিল করতে হলে, নির্ধারিত ফরমে এবং পদ্ধতিতে, আপিলের জন্য নির্দিষ্ট সময়সীমার মধ্যে, যথাযথ ফি দিয়ে, আবেদন দাখিল করতে হবে। আবেদনপত্রের সঙ্গে প্রাসঙ্গিক দলিলাদি সংযুক্ত করতে হবে এবং নির্ধারিত ফরম্যাটে আপিল দাখিলের জন্য নির্দেশনা অনুসরণ করতে হবে। এছাড়াও, আপিলের জন্য নির্দিষ্ট শর্ত ও প্রক্রিয়া অনুসরণ করতে হবে।

**আইনি ভিত্তি:**
আইন অনুযায়ী, ধারা ২৮৭, ২৮৮, এবং ২৮৯ অনুযায়ী, আপিল ট্রাইব্যুনালে আপিলের জন্য নির্দিষ্ট ফরমে, নির্ধারিত সময়ের মধ্যে, এবং প্রয়োজনীয় ফি দিয়ে আবেদন করতে হয়। এছাড়াও, আবেদনপত্রের সঙ্গে প্রাসঙ্গিক দলিলাদি সংযুক্ত করতে হয় এবং নির্ধারিত পদ্ধতি অনুসরণ করতে হয়। 

**বিস্তারিত ব্যাখ্যা:**
(গ) আপিল ট্রাইব্যুনালে আপিল করতে হলে, প্রথমে নির্ধারিত ফরমে আবেদনপত্র পূরণ করতে হবে। এই ফরম বোর্ড বা সংশ্লিষ্ট কর্তৃপক্ষ দ্বারা নির্ধারিত হয় এবং ইলেকট্রনিক বা অন্য কোনো মাধ্যমে দাখিলের ব্যবস্থা থাকতে পারে। আবেদনপত্রের সঙ্গে প্রাসঙ্গিক দলিলাদি যেমন, আপিলের ভিত্তি ও সংশ্লিষ্ট আদেশের সত্যায়িত অনুলিপি সংযুক্ত করতে হবে। 

(খ) আবেদনপত্র দাখিলের জন্য

In [49]:
response = rag_chain.invoke({"input":"আপিল নিস্পত্তির ক্ষেত্রে কি পদ্ধতি অবলম্বন করতে হবে?" })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তর:
আপিল নিস্পত্তির জন্য আপিল ট্রাইব্যুনাল নির্ধারিত তারিখে শুনানি করে, পক্ষদের শুনে এবং প্রয়োজনীয় দলিলাদি ও সাক্ষ্য গ্রহণের মাধ্যমে সিদ্ধান্ত গ্রহণ করে। সিদ্ধান্তের জন্য সংখ্যাগরিষ্ঠ মতামত গ্রহণ করা হয়। যদি মতভেদ হয়, তবে লিখিত সিদ্ধান্ত দেয়া হয়। এছাড়া, বেঞ্চের সদস্যগণ সমানভাবে বিভক্ত হলে প্রেসিডেন্টের নির্দেশে অতিরিক্ত সদস্য নিয়োগ করা হয়। কার্যপদ্ধতি নির্ধারণে আপিল ট্রাইব্যুনাল নিজস্ব নিয়ম তৈরি করতে পারে।

(খ) আইনি ভিত্তি:
ধারা ২৮৬, ২৮৭, ২৮৮, ২৮৯, ২৯০, ২৯১, ২৯২, ২৯৩, ২৯৪, ২৯৫ এবং ২৯৬ অনুযায়ী, আপিলের জন্য নির্ধারিত পদ্ধতি ও সিদ্ধান্ত গ্রহণের প্রক্রিয়া নির্ধারিত হয়েছে। বিশেষ করে, ধারা ২৮৬, ২৮৭, ২৮৮, ২৮৯, ২৯০, ২৯১, ২৯২, ২৯৩, ২৯৪, ২৯৫ এ বিস্তারিত নির্দেশনা রয়েছে।

(গ) বিস্তারিত ব্যাখ্যা:
আপিলের ক্ষেত্রে প্রথমে আপিলকারী নির্ধারিত ফরমে আপিল দাখিল করেন। এরপর, আপিল ট্রাইব্যুনাল তারিখ ও স্থান নির্ধারণ করে নোটিশ দেয়। শুনানির সময় পক্ষগুলো উপস্থিত হয়ে তাদের বক্তব্য উপস্থাপন করে। প্রয়োজন হলে দলিলাদি ও সাক্ষ্য গ্রহণ করা হয়। সিদ্ধান্তের জন্য সংখ্যাগরিষ্ঠ মতামত গ্র

In [16]:
response = rag_chain.invoke({"input":"আয় গোপন করলে কি পরিমান জরিমানা হতে পারে? " })
answer = response.get("answer", "No answer found")
print(answer)

(ক) সংক্ষিপ্ত উত্তর:
আয় গোপন করলে করদাতার উপর ফাঁকি দেওয়া অঙ্কের ১৫% × (১ + ১০% × গ) পরিমান জরিমানা আরোপিত হতে পারে, যেখানে গ হলো অসত্য তথ্য প্রদর্শনের বছর থেকে উদঘাটিত বছর পর্যন্ত মোট বছর সংখ্যা।

(খ) আইনি ভিত্তি:
বাংলাদেশ আয়কর আইনের ধারা ২৭২ অনুযায়ী, আয় গোপন বা অসত্য তথ্য প্রদর্শনের জন্য জরিমানা ধার্য করা হয়।

(গ) বিস্তারিত ব্যাখ্যা:
১. যদি কোনো ব্যক্তি করদাতার প্রদেয় আয়, সম্পদ, দায়, ব্যয়ের তথ্য বা অন্য গুরুত্বপূর্ণ তথ্য অসত্য পরিমাণে প্রদর্শন করে আয় গোপন করেন, তাহলে কার্যক্রম পরিচালনাকারী কর্তৃপক্ষ তার উপর নিম্নলিখিত জরিমানা আরোপ করবে:
 - ক = ফাঁকি দেওয়া অঙ্ক × ১৫%
 - খ = ফাঁকি দেওয়া অঙ্ক × ১০% × গ
 এবং জরিমানার পরিমাণ হবে ক + খ।

২. এখানে,
 - "ফাঁকি দেওয়া অঙ্ক" বলতে সেই করবর্ষের কর ও অন্যান্য অঙ্ক, যেগুলি অসত্য তথ্য প্রদর্শনের কারণে কম প্রদর্শিত হয়েছে,
 - "গ" হল অসত্য তথ্য প্রদর্শনের বছর থেকে সেই তথ্য উদঘাটিত হওয়া পর্যন্ত বছর সংখ্যা।

৩. কার্যক্রম পরিচালনাকারী কর্তৃপক্ষ বলতে উপকর কমিশনার বা তার নিকটস্থ আয়কর কর্তৃপক্ষ ও কর আপিল ট্রাইব্যুনাল বোঝানো হয়।

(ঘ) প্রয়োগ/সত

In [13]:
# Print retrieved context details
context_docs = response.get("context", [])
print(f"📚 Retrieved {len(context_docs)} relevant chunks:")
print(context_docs)

📚 Retrieved 15 relevant chunks:
[Document(id='487d8d47-f58c-49de-bd01-57a9919869d8', metadata={'chunk_id': 'chunk_001', 'chunk_index': 0.0, 'chunk_tokens': 1670.0, 'chunk_type': 'special tax rates on investment', 'keywords': ['বিনিয়োগ', 'বিশেষ কর', 'বিল্ডিং', 'অ্যাপার্টমেন্ট', 'করহার', 'প্লিন্থ আয়তন'], 'part_chapter': 'অংশ ১', 'section_range': 'Section 24 (Reference)', 'source': 'Doc\\11. Income Tax Act, 2023 (22 June 2023)__split__11.txt', 'total_chunks': 26.0}, page_content='পৃষ্ঠা/Page 248\n\n-------------------------------------------------\n\nপ্রথম তফসিল\n\nবিনিয়োগে বিশেষ করহার\n\n[ধারা ২৪ দ্রষ্টব্য]\n\nঅংশ ১\n\nবিশেষ কর প্রদানের মাধ্যমে বিনিয়োগ প্রদর্শন\n\n১। বিশেষ কর প্রদানের মাধ্যমে বিল্ডিং বা অ্যাপার্টমেন্ট বিনিয়োগ প্রদর্শন।—(১) কোনো স্বাভাবিক ব্যক্তি বিল্ডিং বা অ্যাপার্টমেন্ট নির্মাণ বা ক্রয়ে কোনো অর্থ বিনিয়োগ করিলে উক্ত বিনিয়োগকৃত অর্থের উৎস সম্পর্কে ব্যাখ্যা প্রদান করা হইয়াছে বলিয়া গণ্য হইবে, যদি উক্ত বিনিয়োগ সম্পন্ন হওয়া সংশ্লিষ্ট করবর্ষের কর নির্ধারণীর পূর্বে 

In [14]:
# Display just the content in readable format
for i, doc in enumerate(context_docs, 1):
    print(f"\n--- Chunk {i} ---")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content}")
    print("-" * 40)


--- Chunk 1 ---
Source: Doc\11. Income Tax Act, 2023 (22 June 2023)__split__11.txt
Content: পৃষ্ঠা/Page 248

-------------------------------------------------

প্রথম তফসিল

বিনিয়োগে বিশেষ করহার

[ধারা ২৪ দ্রষ্টব্য]

অংশ ১

বিশেষ কর প্রদানের মাধ্যমে বিনিয়োগ প্রদর্শন

১। বিশেষ কর প্রদানের মাধ্যমে বিল্ডিং বা অ্যাপার্টমেন্ট বিনিয়োগ প্রদর্শন।—(১) কোনো স্বাভাবিক ব্যক্তি বিল্ডিং বা অ্যাপার্টমেন্ট নির্মাণ বা ক্রয়ে কোনো অর্থ বিনিয়োগ করিলে উক্ত বিনিয়োগকৃত অর্থের উৎস সম্পর্কে ব্যাখ্যা প্রদান করা হইয়াছে বলিয়া গণ্য হইবে, যদি উক্ত বিনিয়োগ সম্পন্ন হওয়া সংশ্লিষ্ট করবর্ষের কর নির্ধারণীর পূর্বে করদাতা নিম্নবর্ণিত সারণীতে উল্লিখিত হারে কর পরিশোধ করেন:

সারণী

ক্রমিক নং | সম্পত্তির বর্ণনা | করহার
(১) | (২) | (৩)
1. ঢাকার গুলশান মডেল টাউন, বনানী, বারিধারা, মতিঝিল বাণিজ্যিক এলাকা ও দিলকুশা বাণিজ্যিক এলাকায় অবস্থিত অনধিক ২০০ (দুইশত) বর্গমিটার প্লিন্থ আয়তন (plinth area) বিশিষ্ট বিল্ডিং বা অ্যাপার্টমেন্ট | প্রতি বর্গ মিটারে ৪ (চার) হাজার টাকা
2. ঢাকার গুলশান মডেল টাউন, বনানী, বারিধারা, মতিঝিল বাণি

In [ ]:
# # ==========================================
# # STEP 7: Test the Enhanced System
# # ==========================================

# def test_enhanced_rag(query: str):
#     """Test the enhanced RAG system with detailed output"""
#     print(f"\n🔍 Testing Query: '{query}'")
#     print("=" * 60)
    
#     # Get response
#     response = rag_chain.invoke({"input": query})
#     answer = response.get("answer", "No answer found")
    
#     # Print retrieved context details
#     context_docs = response.get("context", [])
#     print(f"📚 Retrieved {len(context_docs)} relevant chunks:")
    
#     for i, doc in enumerate(context_docs[:3]):  # Show first 3
#         metadata = doc.metadata
#         print(f"\nChunk {i+1}:")
#         print(f"  📄 Source: {metadata.get('source', 'Unknown')}")
#         print(f"  📊 Type: {metadata.get('chunk_type', 'Unknown')}")
#         print(f"  🏛️ Act: {metadata.get('act_name', 'Not specified')}")
#         print(f"  📋 Section: {metadata.get('section_range', 'Not specified')}")
#         print(f"  🔤 Keywords: {metadata.get('keywords', [])}")
#         print(f"  📝 Content preview: {doc.page_content[:150]}...")
    
#     print(f"\n🤖 Generated Answer:")
#     print("-" * 40)
#     print(answer)
#     print("=" * 60)

# # Test the system
# test_queries = [
#     "কোম্পানি বলতে কোন কোন সত্তা অন্তর্ভুক্ত?",
#     "আয়কর হার কত?",
#     "পরিচালক নিয়োগের নিয়ম কি?",
# ]

# for query in test_queries:
#     test_enhanced_rag(query)

# print("\n🎉 LLM-based Chunking RAG System Ready!")
# print("✨ Features:")
# print("  - Intelligent legal document chunking")
# print("  - Hierarchical structure preservation") 
# print("  - Enhanced metadata extraction")
# print("  - Context-aware retrieval")
# print("  - Bilingual support (Bangla/English)")

In [24]:
def ask_legal_question(question: str, show_context: bool = False, k: int = 5):
    """
    Simple interface for users to ask legal questions
    
    Args:
        question (str): The legal question in Bangla or English
        show_context (bool): Whether to show retrieved context chunks
        k (int): Number of relevant chunks to retrieve
    
    Returns:
        str: The legal assistant's answer
    """
    
    if not question.strip():
        return "দয়া করে একটি প্রশ্ন লিখুন। / Please enter a question."
    
    try:
        print(f"\n🔍 প্রশ্ন / Question: {question}")
        print("=" * 60)
        
        # Update retriever with new k value if different
        if k != 5:
            global retriever
            retriever = vectorstore.as_retriever(
                search_type="similarity", 
                search_kwargs={'k': k}
            )
        
        # Get response from RAG chain
        response = rag_chain.invoke({"input": question})
        answer = response.get("answer") or response.get("result") or str(response)
        
        # Show context if requested
        if show_context:
            context_docs = response.get("context", [])
            print(f"\n📚 Retrieved {len(context_docs)} relevant chunks:")
            print("-" * 40)
            
            for i, doc in enumerate(context_docs):
                metadata = doc.metadata
                print(f"\nChunk {i+1}:")
                print(f"  📄 Source: {metadata.get('source', 'Unknown')}")
                print(f"  🏛️ Act: {metadata.get('act_name', 'Not specified')}")
                print(f"  📋 Section: {metadata.get('section_range', 'Not specified')}")
                print(f"  📝 Preview: {doc.page_content[:150]}...")
                print("  " + "-" * 35)
        
        print(f"\n🤖 উত্তর / Answer:")
        print("-" * 40)
        print(answer)
        print("=" * 60)
        
        return answer
        
    except Exception as e:
        error_msg = f"❌ Error processing question: {str(e)}"
        print(error_msg)
        return error_msg

def interactive_legal_assistant():
    """
    Interactive mode - continuous question answering
    """
    print("\n🏛️ বাংলাদেশ আইনি সহায়ক / Bangladesh Legal Assistant")
    print("=" * 60)
    print("📝 Instructions:")
    print("  - Ask questions in Bangla or English")
    print("  - Type 'exit' or 'quit' to stop")
    print("  - Type 'context' to show retrieved context")
    print("  - Type 'help' for more commands")
    print("=" * 60)
    
    show_context = False
    
    while True:
        try:
            user_input = input("\n❓ আপনার প্রশ্ন / Your Question: ").strip()
            
            if user_input.lower() in ['exit', 'quit', 'বের হন', 'বন্ধ']:
                print("\n👋 ধন্যবাদ! / Thank you!")
                break
                
            elif user_input.lower() in ['context', 'কনটেক্সট']:
                show_context = not show_context
                status = "ON" if show_context else "OFF"
                print(f"📚 Context display: {status}")
                continue
                
            elif user_input.lower() in ['help', 'সাহায্য']:
                print("\n📋 Available commands:")
                print("  - context: Toggle context display")
                print("  - exit/quit: Exit the assistant")
                print("  - help: Show this help")
                print("  - Just ask any legal question!")
                continue
                
            elif not user_input:
                print("⚠️ Please enter a question.")
                continue
            
            # Process the question
            ask_legal_question(user_input, show_context=show_context)
            
        except KeyboardInterrupt:
            print("\n\n👋 Assistant stopped. ধন্যবাদ! / Thank you!")
            break
        except Exception as e:
            print(f"\n❌ Unexpected error: {e}")

# # Quick test function
# def test_legal_rag():
#     """Test the legal RAG system with sample questions"""
    
#     test_questions = [
#         "কোম্পানি বলতে কোন কোন সত্তা অন্তর্ভুক্ত?",
#         "আয়কর হার কত?",
#         "পরিচালক নিয়োগের নিয়ম কি?",
#         "What is the definition of company?",
#         "Tax rates in Bangladesh"
#     ]
    
#     print("\n🧪 Testing Legal RAG System with Sample Questions:")
#     print("=" * 60)
    
#     for i, question in enumerate(test_questions, 1):
#         print(f"\n🔍 Test {i}: {question}")
#         answer = ask_legal_question(question)
#         print("\n" + "=" * 60)